## I. 패키지 밎 자료 불러오기

In [ ]:
!pip install pymysql
!pip install sqlalchemy
!pip install mysqlclient

     |████████████████████████████████| 43 kB 1.4 MB/s 
     |████████████████████████████████| 87 kB 4.4 MB/s 
  Created wheel for mysqlclient: filename=mysqlclient-2.1.0-cp37-cp37m-linux_x86_64.whl size=99972 sha256=5e19a0948f1e4aff0df77dfa58ab0b8de2851a88f136e5690bfa71bd6f613e9f
  Stored in directory: /root/.cache/pip/wheels/97/d4/df/08cd6e1fa4a8691b268ab254bd0fa589827ab5b65638c010b4
Successfully built mysqlclient


In [ ]:
!pip install pyecharts
!pip install chart_studio

     |████████████████████████████████| 135 kB 8.4 MB/s 
     |████████████████████████████████| 130 kB 54.4 MB/s 
     |████████████████████████████████| 64 kB 2.4 MB/s 


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import time
import datetime
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import pymysql
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc

import itertools
import operator

import base64
from io import BytesIO

from datetime import datetime, timedelta

from pyecharts.charts import Bar
from pyecharts.charts import Line

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
db = pymysql.connect( charset='utf8')
cursor = db.cursor(pymysql.cursors.DictCursor)
sql = 'SELECT * FROM cj'
cursor.execute(sql)
result = cursor.fetchall()
DF = pd.DataFrame(result)


In [ ]:
DF

,index,date,coin_name,s,s.1,lang
0,8,2021-11-22,ada,0,9,kor
1,9,2021-11-22,ada,1,7,kor
2,10,2021-11-22,bch,2,9,kor
3,11,2021-11-22,bch,0,7,kor
4,12,2021-11-22,bch,1,6,kor
5,13,2021-11-22,btc,1,6,kor
6,14,2021-11-22,btc,0,3,kor
7,15,2021-11-22,doge,0,18,kor
8,16,2021-11-22,doge,2,8,kor
9,17,2021-11-22,doge,1,2,kor


In [ ]:
DF.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/sentiment_result_db.csv')

## I. 일단 긍정/중립/부정 3개의 데이터 프래임으로 모으기
- 없을시 0으로 해야 함으로 데이터 프레임을 먼저 만들고 거기에 합치기
- s.1은 갯수

In [ ]:
date = ['2021-11-22','2021-11-23']
# lang = ['kor','eng']
crpyto_name = ['ada','bch','btc','doge','eos','eth','omg','snt','xrp']
s = [0,1,2]



### 1) 새로운 데이터 프레임을 만들어야한다.

In [ ]:
# 새로운 데이터 프레임 만들기
df = pd.DataFrame(columns = ['date', 'crypto_name', 'positive_E','neutrual_E','negative_E','positive_K','neutrual_K','negative_K'])
for i in date:
  for j in crpyto_name:
    df = df.append(pd.DataFrame([[i, j]], columns = ['date','crypto_name']), ignore_index=True)
df

,date,crypto_name,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K
0,2021-11-22,ada,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-11-22,bch,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-11-22,btc,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-11-22,doge,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-11-22,eos,NaN,NaN,NaN,NaN,NaN,NaN
5,2021-11-22,eth,NaN,NaN,NaN,NaN,NaN,NaN
6,2021-11-22,omg,NaN,NaN,NaN,NaN,NaN,NaN
7,2021-11-22,snt,NaN,NaN,NaN,NaN,NaN,NaN
8,2021-11-22,xrp,NaN,NaN,NaN,NaN,NaN,NaN
9,2021-11-23,ada,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# # 연습 빈공간 지정해서 넣기
# for i in range(len(DF)):
#   if (DF['date'][i] == '2021-11-22') and (DF['coin_name'][i] == 'ada') and(DF['lang'][i] == 'kor'):
#     if (DF['s'][i] == 0):
#       print(DF['s.1'][i])
#       a = df.index[(df['date'] == '2021-11-22')&(df['crypto_name'] == 'ada')].tolist()
#       df['positive_K'][a] = DF['s.1'][i]
#     else:
#       print('없음')

9
없음


### 2) 확장 - 모든 코인 모든 날짜

In [ ]:
# for j in date:

#   for i in range(len(DF)):


#     if (DF['date'][i] == j) and (DF['coin_name'][i] == 'ada') and (DF['lang'][i] == 'kor'):
#       if (DF['s'][i] == 0):
#         a = df.index[(df['date'] == j)&(df['crypto_name'] == 'ada')&(DF['lang'][i] == 'kor')].tolist()
#         df['positive_K'][a] = DF['s.1'][i]

In [ ]:
for k in crpyto_name:

  for j in date:

    for i in range(len(DF)):


      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'kor'):
        if (DF['s'][i] == 0):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'kor')].tolist()
          df['positive_K'][a] = DF['s.1'][i]

      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'eng'):
        if (DF['s'][i] == 0):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'eng')].tolist()
          df['positive_E'][a] = DF['s.1'][i]

      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'kor'):
        if (DF['s'][i] == 1):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'kor')].tolist()
          df['neutrual_K'][a] = DF['s.1'][i]

      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'eng'):
        if (DF['s'][i] == 1):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'eng')].tolist()
          df['neutrual_E'][a] = DF['s.1'][i]

      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'kor'):
        if (DF['s'][i] == 2):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'kor')].tolist()
          df['negative_K'][a] = DF['s.1'][i]

      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'eng'):
        if (DF['s'][i] == 2):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'eng')].tolist()
          df['negative_E'][a] = DF['s.1'][i]    

In [ ]:
df

,date,crypto_name,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K
0,2021-11-22,ada,NaN,NaN,NaN,9,7,NaN
1,2021-11-22,bch,NaN,NaN,NaN,7,6,9
2,2021-11-22,btc,NaN,NaN,NaN,3,6,NaN
3,2021-11-22,doge,NaN,NaN,NaN,18,2,8
4,2021-11-22,eos,NaN,NaN,NaN,24,17,13
5,2021-11-22,eth,NaN,NaN,NaN,2,NaN,NaN
6,2021-11-22,omg,NaN,NaN,NaN,2,NaN,NaN
7,2021-11-22,snt,NaN,NaN,NaN,3,NaN,4
8,2021-11-22,xrp,NaN,NaN,NaN,18,2,19
9,2021-11-23,ada,4727,8,22245,3,2,2


In [ ]:
df = df.fillna(0)

In [ ]:
df

,date,crypto_name,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K
0,2021-11-22,ada,0,0,0,9,7,0
1,2021-11-22,bch,0,0,0,7,6,9
2,2021-11-22,btc,0,0,0,3,6,0
3,2021-11-22,doge,0,0,0,18,2,8
4,2021-11-22,eos,0,0,0,24,17,13
5,2021-11-22,eth,0,0,0,2,0,0
6,2021-11-22,omg,0,0,0,2,0,0
7,2021-11-22,snt,0,0,0,3,0,4
8,2021-11-22,xrp,0,0,0,18,2,19
9,2021-11-23,ada,4727,8,22245,3,2,2


## --------- 전처리 작업 완료